# Dealing with NaN

Pandas assigns NaN values to missing data and it's extremely important to deal with this value for our analysis or ML algorithms

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = [pd.Series(np.random.randint(1,50,size=3), index=['mango','papaya', 'avocado']),\
        pd.Series(np.random.randint(1,50,size=4), index=['mango','papaya', 'avocado','kiwi']),\
        pd.Series(np.random.randint(1,50,size=4), index=['mango','papaya', 'avocado', 'kiwi']),\
        pd.Series(np.random.randint(1,50,size=3), index=['mango','papaya', 'kiwi']),\
        pd.Series(np.random.randint(1,50,size=2), index=['avocado','kiwi'])]

In [3]:
df = pd.DataFrame( data, index=['store1', 'store2', 'store3', 'store4', 'store5'])

In [4]:
df

,mango,papaya,avocado,kiwi
store1,7.0,25.0,37.0,NaN
store2,7.0,19.0,39.0,12.0
store3,22.0,48.0,22.0,5.0
store4,36.0,7.0,NaN,8.0
store5,NaN,NaN,42.0,6.0


in cases where we load very large datasets into a DataFrame, possibly with millions of items, the number of NaN values is not easily visualized. For these cases, we can use a combination of methods to count the number of NaN values in our data. 

The methods to use are the isnull and sum

isnull method returns a Boolean DataFrame of the same size as the original df and indicates with True the elements that have NaN values and with False the elements that are not.

In [5]:
df.isnull()

,mango,papaya,avocado,kiwi
store1,False,False,False,True
store2,False,False,False,False
store3,False,False,False,False
store4,False,False,True,False
store5,True,True,False,False


In Pandas, logical True values have numerical value 1 and logical False values have numerical value 0. Therefore, we can count the number of NaN values by counting the number of logical True values. 

We can count them by using the sum method as follows

In [6]:
print(df.isnull().sum())
print(type(df.isnull().sum()))

mango      1
papaya     1
avocado    1
kiwi       1
dtype: int64
<class 'pandas.core.series.Series'>


In [7]:
df.isnull().sum().sum()

4

However another useful approach is counting the total not-NaN values in the DataFrame using the `count()` method which counts for each column the amount of not-NaN values 

In [8]:
df.count()

mango      4
papaya     4
avocado    4
kiwi       4
dtype: int64

then we can sum this Series to have the total amount of not-NaN values in the df

In [9]:
df.count().sum()

16

but the real size of the dataframe is

In [10]:
df.size

20

so to have to know the amount of NaN values we can do 

In [11]:
df.size - df.count().sum()

4

Great! Unfortunately, if we have to work with a NaN-free dataset then we must remove the rows with missing elements.

To drop the rows/columns with NaN values we must use the `dropna(axis=0|1)` 

**Note**

the original DataFrame is not modified. To allow drop method to modify the dataset then we must add the `inplace = True` property

Rows

In [12]:
df.dropna(axis=0)

,mango,papaya,avocado,kiwi
store2,7.0,19.0,39.0,12.0
store3,22.0,48.0,22.0,5.0


Columns

In [13]:
df.dropna(axis=1)

""
store1
store2
store3
store4
store5


In [14]:
df1 = df.copy()
df1.dropna(axis=1, inplace=True)
df1

""
store1
store2
store3
store4
store5


In [15]:
df

,mango,papaya,avocado,kiwi
store1,7.0,25.0,37.0,NaN
store2,7.0,19.0,39.0,12.0
store3,22.0,48.0,22.0,5.0
store4,36.0,7.0,NaN,8.0
store5,NaN,NaN,42.0,6.0


However, drop an entire row/column for few NaN may be too greeady as approach. Instead of dropping this kind of data, we can replace NaN values with with forward filling, we can use 0 or the previous values taken from columns or rows. The .fillna(method = 'ffill', axis) will use the forward filling (ffill) method to replace NaN values using the previous known value along the given axis

**Note**

The .fillna() method replaces (fills) the NaN values out of place. This means that the original DataFrame is not modified. You can always replace the NaN values in place by setting the keyword inplace = True inside the fillna() function.

In [16]:
df

,mango,papaya,avocado,kiwi
store1,7.0,25.0,37.0,NaN
store2,7.0,19.0,39.0,12.0
store3,22.0,48.0,22.0,5.0
store4,36.0,7.0,NaN,8.0
store5,NaN,NaN,42.0,6.0


In [17]:
df.fillna(0)

,mango,papaya,avocado,kiwi
store1,7.0,25.0,37.0,0.0
store2,7.0,19.0,39.0,12.0
store3,22.0,48.0,22.0,5.0
store4,36.0,7.0,0.0,8.0
store5,0.0,0.0,42.0,6.0


In [18]:
df.fillna(method = 'ffill', axis = 0)

,mango,papaya,avocado,kiwi
store1,7.0,25.0,37.0,NaN
store2,7.0,19.0,39.0,12.0
store3,22.0,48.0,22.0,5.0
store4,36.0,7.0,22.0,8.0
store5,36.0,7.0,42.0,6.0


In [19]:
df.fillna(method = 'ffill', axis = 1) 

,mango,papaya,avocado,kiwi
store1,7.0,25.0,37.0,37.0
store2,7.0,19.0,39.0,12.0
store3,22.0,48.0,22.0,5.0
store4,36.0,7.0,7.0,8.0
store5,NaN,NaN,42.0,6.0


We can notice that the we have some limit cases:
    - the element kiwi for store1 
    - the element mango for store3
    
the previous methods don't cover both the methods because the didn't have a previous element in the row or column. 

A possibile solution is to use the combination of this method when axis=0 and axis=1

In [20]:
df.fillna(method = 'ffill', axis = 1).fillna(method = 'ffill', axis = 0)

,mango,papaya,avocado,kiwi
store1,7.0,25.0,37.0,37.0
store2,7.0,19.0,39.0,12.0
store3,22.0,48.0,22.0,5.0
store4,36.0,7.0,7.0,8.0
store5,36.0,7.0,42.0,6.0


An altertative to the ffill method is the backfill one which badkword filling the missing values

In [21]:
df.fillna(method = 'backfill', axis = 0)

,mango,papaya,avocado,kiwi
store1,7.0,25.0,37.0,12.0
store2,7.0,19.0,39.0,12.0
store3,22.0,48.0,22.0,5.0
store4,36.0,7.0,42.0,8.0
store5,NaN,NaN,42.0,6.0


In [22]:
df.fillna(method = 'backfill', axis = 1)

,mango,papaya,avocado,kiwi
store1,7.0,25.0,37.0,NaN
store2,7.0,19.0,39.0,12.0
store3,22.0,48.0,22.0,5.0
store4,36.0,7.0,8.0,8.0
store5,42.0,42.0,42.0,6.0


The last but not latest method is the interpolate function that allows us to replace NaN values by using different interpolation methods.

a list of possibile interpolation methods is provided by the [official doc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.interpolate.html)

In [23]:
df.interpolate(method='linear', axis=0)

,mango,papaya,avocado,kiwi
store1,7.0,25.0,37.0,NaN
store2,7.0,19.0,39.0,12.0
store3,22.0,48.0,22.0,5.0
store4,36.0,7.0,32.0,8.0
store5,36.0,7.0,42.0,6.0


In [24]:
df.interpolate(method='linear', axis=1)

,mango,papaya,avocado,kiwi
store1,7.0,25.0,37.0,37.0
store2,7.0,19.0,39.0,12.0
store3,22.0,48.0,22.0,5.0
store4,36.0,7.0,7.5,8.0
store5,NaN,NaN,42.0,6.0
